In [10]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
from functools import partial
import geopy
import time
from geopy.extra.rate_limiter import RateLimiter
from opencage.geocoder import OpenCageGeocode
from pprint import pprint

In [11]:
URL = "https://www.cntraveler.com/galleries/2015-11-27/the-50-most-beautiful-places-in-the-world"

In [12]:
# Header for request
HEADER = ({'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en; q=0.5'})
geolocator = Nominatim(user_agent="geocode_locations_from_webpage")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1.5, max_retries=3)
geo_key = 'e0e73007ebe240c6836e278bbc510da2'
geocoder = OpenCageGeocode(geo_key)

In [13]:
webpage = requests.get(URL, headers=HEADER)
webpage

<Response [200]>

In [14]:
soup = BeautifulSoup(webpage.content, "html.parser")
soup

<!DOCTYPE html>
<html data-reactroot="" lang="en-US"><head><title>The 51 Most Beautiful Places in the World | Condé Nast Traveler</title><meta charset="utf-8"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><meta content="no" name="msapplication-tap-highlight"/><meta content="width=device-width, initial-scale=1" name="viewport"/><meta content="no-referrer-when-downgrade" name="referrer"/><meta content="Condé Nast" name="author"/><meta content="Copyright (c) Condé Nast 2023" name="copyright"/><meta content="Pinpointing all of the most beautiful places in the world could take a lifetime—but these 51 destinations, from the Serengeti to the Grand Canyon, will keep you plenty busy." name="description"/><meta content="56549c695e7aeb7a6e217614" name="id"/><meta content="islands,outdoors,national parks" name="keywords"/><meta content="islands,outdoors,national parks" name="news_keywords"/><meta content="index, follow, max-image-preview:large" name="robots"/><meta content="gallery" name=

In [22]:
places_list = soup.find_all("span", attrs = {'class': 'GallerySlideCaptionHedText-iqjOmM jwPuvZ'})
#print(len(places))
#print(places[4].text)
#places.pop(4)
#print(len(places))
#print(places[4].text)
places_list[0:10]

[<span class="GallerySlideCaptionHedText-iqjOmM jwPuvZ">The Azores, Portugal</span>,
 <span class="GallerySlideCaptionHedText-iqjOmM jwPuvZ">Mount Kilimanjaro, Tanzania</span>,
 <span class="GallerySlideCaptionHedText-iqjOmM jwPuvZ">Wulingyuan Scenic Area, China</span>,
 <span class="GallerySlideCaptionHedText-iqjOmM jwPuvZ">Arashiyama Bamboo Grove, Japan</span>,
 <span class="GallerySlideCaptionHedText-iqjOmM jwPuvZ">Antarctica</span>,
 <span class="GallerySlideCaptionHedText-iqjOmM jwPuvZ">Na Pali Coast, Hawaii</span>,
 <span class="GallerySlideCaptionHedText-iqjOmM jwPuvZ">Namib Desert, Namibia</span>,
 <span class="GallerySlideCaptionHedText-iqjOmM jwPuvZ">Reynisfjara, Iceland</span>,
 <span class="GallerySlideCaptionHedText-iqjOmM jwPuvZ">Capri, Italy</span>,
 <span class="GallerySlideCaptionHedText-iqjOmM jwPuvZ">Cappadocia, Turkey</span>]

In [26]:
places = []
places_list.pop(4)
l = len(places_list)
l
for i in range(11):
    places.append(places_list[i].text)

places

['The Azores, Portugal',
 'Mount Kilimanjaro, Tanzania',
 'Wulingyuan Scenic Area, China',
 'Arashiyama Bamboo Grove, Japan',
 'Capri, Italy',
 'Cappadocia, Turkey',
 'Angel Falls, Venezuela',
 'Galápagos Islands, Ecuador',
 'Cliffs of Moher, Ireland',
 'The Great Barrier Reef, Australia',
 'The Grand Canyon, Arizona']

In [ ]:
town,city,state = "","",""
location_description = geocoder.geocode(places[4].text)
try:
    town = location_description[0]['components']['town']
except:
    print ("No data")
try:
    city = location_description[0]['components']['city']
except:
    print ("No data")
try:
    state = location_description[0]['components']['state']
except:
    print ("No data")

print( town + " " + city + "  " + state + "  ")
#location_description

In [ ]:
city_address = []
for i in range(20):
    
    # GETTING RAW DATA
    try:
        location_description = geocoder.geocode(places[i].text)
    except Exception:
        print(str(i) + " " + "No data found for location: " + places[i].text)
        continue
    
    # GETTING ADDRESS VALUES
    latitude = str(location_description[0]['geometry']['lat'])
    longitude = str(location_description[0]['geometry']['lng'])
    loc_title = places[i].text.split(",")[0]
    
    try:
        city = location_description[0]['components']['city']
    except Exception:
        try:
            city = location_description[0]['components']['town']
        except Exception:
            try:
                city = location_description[0]['components']['state']
            except Exception:
                city = "NULL"
    country = location_description[0]['components']['country']
    country_code = location_description[0]['components']['country_code']
    continent = location_description[0]['components']['continent']
    
    #GETTING POSTCODE
    lat_long = str(latitude) + ","+ str(longitude) 
    location2 = geolocator.reverse(lat_long, language="en")
    try:
        postcode = location2.raw['address']['postcode'].replace(" ","")
    except:
        postcode = "NULL"
        
        
    # ADDING VARIABLES TO DICTIONARY
    #print(str(i) +"  "+ loc_title + "  " + city +"  " + country + "  " + country_code.upper() + "  " + continent + "  " + latitude + "  "+ longitude+ "  " + postcode)
    address_dict = {'Location': loc_title, 'City/State': city, 'Country': country, 'Country Code':country_code.upper(), 'Continent': continent, 'Latitude': str(round(float(latitude),2)), 'Longitude': str(round(float(longitude),2)), 'Postcode': postcode}
    city_address.append(address_dict)
    
print(city_address)

In [ ]:
ca_most_beautiful_df = pd.DataFrame.from_dict(city_address)
ca_most_beautiful_df